In [2]:
import numpy as np
from scipy.stats import norm


def portfolio_projection(C, N, mu, sigma, percentiles=[10, 50, 90]):
    """
    Calculate the p-th percentiles of the N-year accumulated value W
    when investing a fixed amount C at the end of each year,
    with returns ~ N(mu, sigma^2), using closed-form mean-variance lognormal approximation.

    Parameters:
    - C: float, annual contribution
    - N: int, number of years
    - mu: float, expected return per year (e.g., 0.06 for 6%)
    - sigma: float, volatility per year (e.g., 0.15 for 15%)
    - percentiles: list of floats, desired percentiles (0-100)

    Returns:
    - dict mapping each percentile to the projected value
    """
    # Building blocks
    m = 1 + mu
    A = m**2 + sigma**2

    # Mean of W
    E = C * (m**N - 1) / (m - 1)
    # Second moment of W
    S = (A**N - 1) / (A - 1)
    T = sum((m**p) * ((A ** (N - p) - 1) / (A - 1)) for p in range(1, N))
    second_moment = C**2 * (S + 2 * T)

    # Variance of W
    V = second_moment - E**2

    # Lognormal approximation parameters
    sigma_w2 = np.log(1 + V / E**2)
    sigma_w = np.sqrt(sigma_w2)
    mu_w = np.log(E) - 0.5 * sigma_w2

    # Compute percentiles
    results = {}
    for p in percentiles:
        if p == 50:
            results[p] = E  # use mean for median
        else:
            z = norm.ppf(p / 100)
            results[p] = np.exp(mu_w + z * sigma_w)

    return results

In [3]:
def base_case_projection(C, N, mu):
    """
     N-year portfolio projection with
    annual contributions at end of each year and normally-distributed returns.
    """
    # Initialize an array to store the portfolio value at the end of each year.
    # We start with a value of 0 at year 0.
    portfolio_path = np.zeros(N + 1)

    # Loop through each year to calculate the new portfolio value.
    for year in range(1, N + 1):
        # The new value is the previous year's value grown by the return,
        # plus the new contribution for the current year.
        portfolio_path[year] = portfolio_path[year - 1] * (1 + mu) + C

    # Return the path, excluding the initial value at year 0.
    return portfolio_path[1:]

In [4]:
import numpy as np


def mc_portfolio_projection(C, N, mu, sigma, num_sims=100000, seed=None):
    """
    Monte Carlo simulation for N-year portfolio projection with
    annual contributions at end of each year and normally-distributed returns.

    Parameters:
    - C: float, annual contribution
    - N: int, number of years
    - mu: float, expected return per year (e.g., 0.06 for 6%)
    - sigma: float, volatility per year (e.g., 0.15 for 15%)
    - num_sims: int, number of simulation paths
    - seed: int or None, random seed for reproducibility

    Returns:
    - dict with keys 'mean', '10th_percentile', '90th_percentile'
    """
    if seed is not None:
        np.random.seed(seed)

    # Simulate returns: shape (num_sims, N)
    returns = np.random.normal(mu, sigma, size=(num_sims, N))
    growth = 1 + returns

    # Initialize array for portfolio value
    portfolio = np.zeros(num_sims)

    # For each year: apply return, then contribute C at end of year
    for year in range(N):
        portfolio *= growth[:, year]
        portfolio += C

    # Compute statistics
    mean = portfolio.mean()
    pct10 = np.percentile(portfolio, 10)
    pct90 = np.percentile(portfolio, 90)

    return {"mean": mean, "10th_percentile": pct10, "90th_percentile": pct90}

In [13]:
C = 300_000
N = 25
mu = 0.06
sigma = 0.10
percentiles = [10, 50, 90]
num_sims = 200_000
seed = 42

res = portfolio_projection(C=C, N=N, mu=mu, sigma=sigma, percentiles=percentiles)
res

{10: 10454684.849671938, 50: 16459353.598717427, 90: 23460038.819853365}

In [15]:
res[10] + 60_000 * 25, res[90] + 60_000 * 25

(11954684.849671938, 24960038.819853365)

In [9]:
res = mc_portfolio_projection(
    C=C, N=N, mu=mu, sigma=sigma, num_sims=num_sims, seed=seed
)
res

{'mean': 16445829.165690389,
 '10th_percentile': 10515826.617756704,
 '90th_percentile': 23417944.79658501}

In [11]:
res = base_case_projection(C=C, N=N, mu=mu)
res[-1]

16459353.598717425